In [374]:
#https://reverb.com/page/reverb-data-engineering-coding-challenge
import pandas as pd
import numpy as np
import urllib
import re

data = urllib.request.urlopen('https://gist.githubusercontent.com/mrgaaron/843feea23fc14621a0b5085dc6e81e73/raw/af9bea1e9a570c300699036c862604af7a5977e4/gistfile1.txt')
content = data.read().decode('ascii')

In [375]:
temp = []
rgx = re.compile(r'.*url": "(.*)", "timestamp": "(.*)", "sessionId": "(.*)", "experiments": "(.*)", "userId": (.*)}')
for line in content.split('\n'):
    a=rgx.match(line).groups()
    temp.append(a)
log = pd.DataFrame(temp,columns=['url','timestamp','sessionId','experiments','userId'])
log['timestamp'] = pd.to_datetime(log['timestamp'])
log.head()

,url,timestamp,sessionId,experiments,userId
0,/orchestra,2018-01-02 15:41:13.620579,6d707042-801a-41de-9406-54e46f6ae433,full-screen-ads,4
1,/marketplace,2018-01-02 15:41:17.620579,ecfb723c-b423-4644-ace8-a3d518c2af50,full-screen-ads,8
2,/orchestra,2018-01-02 15:41:19.620579,b1c15529-f3f0-461c-b2dd-b60fb02223c3,full-screen-ads,2
3,/pedals,2018-01-02 15:41:22.620579,cbb2aa48-24d9-4bc3-bd86-75072f75096d,no-ads,3
4,/bass-guitars,2018-01-02 15:41:24.620579,6d707042-801a-41de-9406-54e46f6ae433,full-screen-ads,4


In [363]:
top5 = list(log.groupby(['url']).url.agg({"count": len}).nlargest(5,columns = ['count']).reset_index().url)
top5

C:\Users\animv\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: FutureWarning: using a dict on a Series for aggregation
is deprecated and will be removed in a future version
  """Entry point for launching an IPython kernel.


['/orchestra', '/pro-audio', '/pedals', '/bass-guitars', '/marketplace']

In [369]:
grouped = log.groupby(['userId','sessionId'],as_index=False)
newdf = grouped['timestamp'].min()
newdf['min_time'] = newdf['timestamp']
newdf = newdf.drop('timestamp',axis=1)
newdf['max_time'] = grouped['timestamp'].max()['timestamp']
newdf['session'] = (pd.DatetimeIndex(newdf.max_time).astype ( np.int64 )/1000000 - pd.DatetimeIndex(newdf.min_time).astype ( np.int64 )/1000000)
newdf['session1'] = newdf.max_time - newdf.min_time
newdf=newdf.drop(['max_time','min_time'],axis=1)
newdf.head()

,userId,sessionId,session,session1
0,1,0fe8e0c7-65d8-48e4-b33a-c85f10986251,21000.0,00:00:21
1,1,24ac40e8-d84b-44f1-8ef0-6538ab101db7,69000.0,00:01:09
2,1,3e653ab9-2de8-4915-bfe7-9184613b7317,68000.0,00:01:08
3,1,6c8de33e-a1ad-4f77-9de3-93663e00ec7e,24344000.0,06:45:44
4,1,72843ddd-489d-4285-b12c-45a7831a1ed8,0.0,00:00:00


In [365]:
stats  = {}
stats['min'] = newdf['session'].min()
stats['max'] = newdf['session'].max()
stats['mean'] = newdf['session'].mean()
stats['median'] = newdf['session'].median()
stats

{'max': 24655000.0, 'mean': 2269928.5714285714, 'median': 27500.0, 'min': 0.0}

In [366]:
dict1 = {'most_viewed_urls' : top5}
dict2 = {'session_stats': stats}
{**dict1, **dict2}

{'most_viewed_urls': ['/orchestra',
  '/pro-audio',
  '/pedals',
  '/bass-guitars',
  '/marketplace'],
 'session_stats': {'max': 24655000.0,
  'mean': 2269928.5714285714,
  'median': 27500.0,
  'min': 0.0}}

In [367]:
#Extra

new_group = newdf.groupby(['userId'],as_index=False)
op = new_group['session'].mean()
op['mean'] = op['session'] 
op=op.drop(['session'],axis=1)
op['median'] = new_group['session'].median()['session']
op['min'] = new_group['session'].min()['session']
op['max'] = new_group['session'].max()['session']
op

,userId,mean,median,min,max
0,1,3.094875e+06,68500.0,0.0,24344000.0
1,2,1.446706e+06,18000.0,0.0,24030000.0
2,3,2.056000e+06,13000.0,0.0,24128000.0
3,4,2.473600e+06,11000.0,0.0,24531000.0
4,5,4.142000e+06,63000.0,0.0,24655000.0
5,6,1.755286e+06,68500.0,0.0,23545000.0
6,7,2.252455e+06,24000.0,0.0,24475000.0
7,8,1.900769e+06,25000.0,0.0,24264000.0
8,9,3.539857e+06,3000.0,0.0,24487000.0
